### This Notebook is an attempt to create a GUI through Python.
**I am going to be sincere and say that this work is all sweat and tears from Chat GPT**

In [ ]:
## ideas to incorporate!
## ctrl z button
## a next button: you load a file with a number of profiles in it and then you can flick through them one by one 


In [7]:
import tkinter as tk
from tkinter import filedialog
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

In [8]:
# Generate dummy data for temperature profiles
num_profiles = 5
depth = np.arange(0, 1000, 10)  # Depth in meters
temperature_profiles = []
for _ in range(num_profiles):
    temperature = np.random.uniform(2, 20, len(depth))  # Temperature in °C
    temperature_profiles.append((depth, temperature))


In [ ]:
%matplotlib qt

class OceanProfileGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Ocean Profile Analysis")

        self.load_button = tk.Button(root, text="Load Profile", command=self.load_profile)
        self.load_button.pack()

        self.canvas = plt.figure(figsize=(6, 4))
        self.plot_area = self.canvas.add_subplot(111)
        self.plot_area.set_xlabel("Temperature (°C)")
        self.plot_area.set_ylabel("Depth (m)")

        self.canvas_widget = FigureCanvasTkAgg(self.canvas, master=root)
        self.canvas_widget.get_tk_widget().pack()

        self.selected_depth = None
        self.canvas_widget.mpl_connect('button_press_event', self.on_click)

    def load_profile(self):
        file_path = filedialog.askopenfilename(title="Select Ocean Temperature Profile")
        # Load temperature profile from file and plot it
        # Replace this with your actual code to load and plot the profile

    def on_click(self, event):
        # Get depth where user clicked
        depth = event.ydata
        # Record selected depth
        self.selected_depth = depth
        # Mark selected depth on the plot
        self.plot_area.axhline(depth, color='r', linestyle='--')
        self.canvas_widget.draw()

if __name__ == "__main__":
    root = tk.Tk()
    gui = OceanProfileGUI(root)
    root.mainloop()
